# Install dependencies

In [ ]:
!pip install -q mlflow==2.20.4 boto3 scikit-learn

# Run a minimal example with MLflow logging enabled
This example comes straight from the [MLflow docs](https://mlflow.org/docs/latest/getting-started/intro-quickstart/index.html).  
    
`mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")` as in the example is not needed, as mlflow uses `MLFLOW_TRACKING_URI` variable by default. This variable is set in the `mlflow` PodDefault. However, if MLflow is not configured for the namespace, the tracking URI will not be set, and the experiment will be saved locally.

In [ ]:
import mlflow
import os

In [ ]:
if mlflow.get_tracking_uri().startswith("file://") or "MLFLOW_TRACKING_URI" not in os.environ:
    print("\033[93m⚠️  WARNING:\033[0m No MLflow server configured — experiments will be logged locally in `mlruns/`")
    input("Press any key to proceed...")

In [ ]:
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)

# Create a new MLflow Experiment
mlflow.set_experiment("MLflow Quickstart")

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")
    
    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )